In [1]:
from train_model import *

In [5]:
import json

In [9]:
path_to_config = 'models/0308_env_aspp_mlp_backbone/config.json'
with open(path_to_config, "r") as f:
    config = json.load(f)
config = {k: v if v != "" else None for k,v in config.items()}
model_setup = {}
if config['env_model'] is not None: 
    config['env_model']['covariates'] = [eval(f) for f in config['env_model']['covariates']]
    model_setup['env'] = config['env_model']
if config['sat_model'] is not None: 
    config['sat_model']['covariates'] = [eval(f) for f in config['sat_model']['covariates']]
    model_setup['sat'] = config['sat_model']
train_occ_path = eval(config['train_occ_path'])
random_bg_path = eval(config['random_bg_path']) if config['random_bg_path'] is not None else None
val_occ_path = eval(config['val_occ_path'])
n_max_low_occ = config['n_max_low_occ']
embed_shape = config['embed_shape']
loss = config['loss']
lambda2 = config['lambda2']
batch_size = config['batch_size']

In [10]:
providers = []
for model_dict in model_setup.values():
    flatten = True if model_dict['model_name'] == 'MLP' else False 
    providers.append(make_providers(
        model_dict['covariates'], model_dict['patch_size'], flatten
    ))


Making patch providers with size=1x1, flatten=False for covariates:
	 - data/full_data/EnvironmentalRasters/Climate/BioClimatic_Average_1981-2010/
	 - data/full_data/EnvironmentalRasters/Soilgrids/
	 - data/full_data/EnvironmentalRasters/LandCover/LandCover_MODIS_Terra-Aqua_500m.tif


KeyboardInterrupt: 

In [ ]:
train_data = PatchesDatasetCooccurrences(
    occurrences=train_occ_path, 
    providers=providers, 
    pseudoabsences=random_bg_path, 
    n_low_occ=n_max_low_occ
)

In [ ]:
model_setup = {}
if dict['env_model'] is not None: model_setup['env'] = config['env_model']
if dict['sat_model'] is not None: model_setup['sat'] = config['sat_model']
model_setup

In [ ]:
[eval(f) for f in config['env_model']['covariates']]

In [ ]:
config['e_model']

In [ ]:
train_occ_path=po_path_sampled_25
random_bg_path=None
val_occ_path=pa_path
env_model={'model_name':'ASPP', 'covariates':[bioclim_dir, soil_dir, landcover_path], 'patch_size': 10, 'embed_shape': 64, 'kernel_sizes': [1,3,3], 'dilations': [1,1,2]}
sat_model=None
n_max_low_occ=50
embed_shape=None
loss='an_full_loss'
lambda2=None
n_epochs=100
batch_size=128
learning_rate=1e-3
seed=42

In [ ]:
model_setup = {}
if env_model is not None: model_setup['env'] = env_model
if sat_model is not None: model_setup['sat'] = sat_model
print(model_setup)

In [ ]:
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"DEVICE: {dev}")

In [ ]:
train_data, val_data, model, optimizer, multires = setup_model(
    model_setup, train_occ_path, random_bg_path, val_occ_path, n_max_low_occ,
    embed_shape, learning_rate, seed)
model = model.to(dev)

In [ ]:
batch_size=10
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(val_data, shuffle=False, batch_size=batch_size)#, num_workers=4)

In [ ]:
loss_fn = eval(loss)
species_weights = torch.tensor(train_data.species_weights).to(dev)
val_loss_fn = torch.nn.BCELoss()

In [ ]:
for po_inputs, bg_inputs, labels in tqdm(train_loader):
   labels = labels.to(torch.float32).to(dev)
   inputs = po_inputs[0].to(torch.float32).to(dev)
   break

In [ ]:
y_pred = torch.sigmoid(model(inputs))

In [ ]:
y_pred.shape

In [ ]:

x = inputs
x_in = [x[:, :, imin:imax, imin:imax] for imin, imax in zip(model.imins, model.imaxs)]

In [ ]:
x_in[0].shape

In [ ]:
m = model.aspp_branches[0].to(dev)
m(x_in[0])

In [ ]:

x = [aspp(xi) for aspp, xi in zip(model.aspp_branches, x_in)]

In [ ]:
print([xi.get_device() for xi in x_in])

In [ ]:
print(model.subpatch_sizes)
print(model.center_idx)
print(model.imins)
print(model.imaxs)

In [ ]:
model.center_idx

In [ ]:
[i >= 0 for i in imins]

In [ ]:
(np.array(imaxs) < 4).all()

In [ ]:
    y_pred = torch.sigmoid(model(inputs))


In [ ]:
from torch import nn

def aspp_branch(in_channels, out_channels, kernel_size, dilation):
    '''
    As implemented in:
    https://github.com/yassouali/pytorch-segmentation/blob/master/models/deeplabv3_plus.py
    '''
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size,dilation=dilation),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True))

In [ ]:
kernel_sizes = [1, 3, 3, 3]
dilations = [1, 1, 2, 3]
subpatch_sizes = [(d-1)*(k-1) + k for k, d in zip(kernel_sizes, dilations)]
print(subpatch_sizes)
in_patch_size = 8
center_idx = in_patch_size // 2
print(center_idx)

In [ ]:
in_channels = 41
out_channels = 64
aspp_branches = [aspp_branch(in_channels, out_channels, k, d) for k, d in zip(kernel_sizes, dilations)]
len(aspp_branches)

In [ ]:
x = po_inputs[0].to(torch.float32)
imins = [int(center_idx - (s-1)/2) for s in subpatch_sizes]
imaxs = [int(center_idx + (s-1)/2 + 1) for s in subpatch_sizes]
x_in = [x[:, :, imin:imax, imin:imax] for imin, imax in zip(imins, imaxs)]
x_out = [aspp(xi) for aspp, xi in zip(aspp_branches, x_in)]

In [ ]:
print([o.shape for o in x_out])

In [ ]:
x = torch.cat(x_out, dim=1).squeeze()
print(x.shape)

In [ ]:
l = nn.Linear(256, 10)
l(x).shape

In [ ]:
kernel_sizes = [1, 3, 3, 3]
dilations = [1, 1, 2, 3]
subpatch_sizes = [(d-1)*(k-1) + k for k, d in zip(kernel_sizes, dilations)]
print(subpatch_sizes)
in_patch_size = 9
center_idx = in_patch_size // 2
print(center_idx)

In [ ]:
imins = [int(center_idx - (s-1)/2) for s in subpatch_sizes]
imaxs = [int(center_idx + (s-1)/2 + 1) for s in subpatch_sizes]
x_in = [x[:, imin:imax, imin:imax] for imin, imax in zip(imins, imaxs)]
print([y.shape for y in x_in])

In [ ]:
in_channels = 41
out_channels = 128
aspp_branches = [aspp_branch(in_channels, out_channels, k, d) for k, d in zip(kernel_sizes, dilations)]

In [ ]:
xi = x_in[0].to(torch.double)
aspp = aspp_branches[0]
print(xi.shape)
print(aspp)

In [ ]:
import pdb
pdb.set_trace()
aspp(xi)

In [ ]:
x_out = [torch.rand((out_channels, 1, 1)) for i in range(5)]

In [ ]:
torch.cat(x_out, dim=0).shape

In [ ]:
item = train_data.items.iloc[0]

In [ ]:
print(f"{p.n_rows} - ({item.lat} - {p.y_min}) / {p.y_resolution})")
print(p.n_rows - (item.lat - p.y_min) / p.y_resolution)
x = int(p.n_rows - (item.lat - p.y_min) / p.y_resolution)
y = int((item.lon - p.x_min) / p.x_resolution)
print(x, y)

In [ ]:
print(f"{x} - ({p.patch_size} // 2), {x} + ({p.patch_size} // 2)")
print(x - (p.patch_size // 2), x + (p.patch_size // 2))

In [ ]:
print(f"{y} - ({p.patch_size} // 2), {y} + ({p.patch_size} // 2)")
print(y - (p.patch_size // 2), y + (p.patch_size // 2))

In [ ]:
patch_data = [p.data[i, x - (p.patch_size // 2): x + (p.patch_size // 2), y - (p.patch_size // 2): y + (p.patch_size // 2)] for i in range(p.nb_layers)]


In [ ]:
patch_data[0].shape

In [ ]:
x = np.array([1,2,3,4,5,7,5,8,4,7,3,67,3,2,8,3,5,7,2,6,5,6])
x[3:8]

In [ ]:
4511-4447

In [ ]:
for i, key in enumerate(model_setup.keys()):
    model_setup[key]['input_shape'] = train_data[0][0][i].shape
    if multires:
        assert embed_shape is not None
        model_setup[key]['output_shape'] = embed_shape
    else:
        model_setup[key]['output_shape'] = train_data.n_species
print([[m['input_shape'], m['output_shape']] for m in model_setup.values()])

In [ ]:
val_data = PatchesDatasetCooccurrences(
    occurrences=val_occ_path,
    providers=providers, 
    species=train_data.species, 
    n_low_occ=n_max_low_occ
)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(val_data, shuffle=False, batch_size=batch_size)#, num_workers=4)

In [ ]:
model_list = [make_model(model_dict) for model_dict in model_setup.values()]
if multires:
    model = MultiScaleModel(
        model_list[0], model_list[1], train_data.n_species, embed_shape, embed_shape
    ).to(dev)
else:
    model = model_list[0].to(dev)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# loss functions
loss_fn = eval(loss)
species_weights = torch.tensor(train_data.species_weights).to(dev)
val_loss_fn = torch.nn.BCELoss()

In [ ]:
arch ='resnet18'
block = BasicBlock
layers = [2, 2, 2, 2]
pretrained = False# 'ResNet18_Weights.IMAGENET1K_V1' 
progress = True

model = ResNet(block, layers, **kwargs)
    # if pretrained:
    #     state_dict = load_state_dict_from_url(model_urls[arch],
    #                                           progress=progress)
    #     model.load_state_dict(state_dict)

In [ ]:
for po_inputs, bg_inputs, labels in tqdm(train_loader):
    labels = labels.to(torch.float32).to(dev) 
    print(labels.shape)

    if multires:
        inputsA = po_inputs[0].to(torch.float32).to(dev)
        inputsB = po_inputs[1].to(torch.float32).to(dev)
        y_pred = torch.sigmoid(model(inputsA, inputsB))

    else:
        if random_bg_path is None:
            inputs = po_inputs[0].to(torch.float32).to(dev)
            print(po_inputs[0].shape)
        else:
            inputs = torch.cat((po_inputs[0], bg_inputs[0]), 0).to(torch.float32).to(dev)
            print(po_inputs[0].shape, len(po_inputs))
            print(len(bg_inputs), bg_inputs[0].shape)
            print(inputs.shape)
        y_pred = torch.sigmoid(model(inputs))
            
    if random_bg_path is None:
       train_loss = loss_fn(y_pred, labels, species_weights)
          
    else:
        po_pred = y_pred[0:len(po_inputs[0])]
        bg_pred = y_pred[len(po_inputs[0]):]        
        train_loss = loss_fn(po_pred, labels, species_weights, lambda2, bg_pred)

    print(train_loss)
    break

In [ ]:
torch.sigmoid(model_list[0](inputsA)).min()

In [ ]:
torch.sigmoid(model_list[1](inputsB)).min()

In [ ]:
po_inputs[0].shape

In [ ]:
    
model = get_model(model, input_shape[0], n_species, model_params, patch_size).to(dev)


In [ ]:
flatten = [True if model == 'MLP' else False for model in models]
flatten

In [ ]:
providers = []
for group, patch_size, f in zip(covariates, patch_sizes, flatten):

    group_providers = []
    for cov in group:
        print(f"\t - {cov}")
        if 'SatelliteImages' in cov:
            if f and patch_size != 1: 
                exit("jpeg patch provider for satellite images cannot flatten image patches")
            group_providers.append(JpegPatchProvider(cov, size=patch_size))
        elif '.tif' in cov:
            group_providers.append(RasterPatchProvider(cov, size=patch_size, flatten=f))
        else:
            group_providers.append(MultipleRasterPatchProvider(cov, size=patch_size, flatten=f))

    providers.append(group_providers)

In [ ]:
train_data = MultiScalePatchesDatasetCooccurrences(occurrences=train_occ_path, providersA=providers[0], providersB=providers[1])

In [ ]:
tmp = train_data[0]

In [ ]:
index = 0
item = train_data.items.iloc[index][train_data.item_columns].to_dict()
item_species = train_data.items.iloc[index][train_data.label_name]
labels = 1 * np.isin(train_data.species, item_species)
labels

In [ ]:
next(iter(train_loader))

In [ ]:
tmp = train_data[0]

In [ ]:
tmp[1].shape

In [ ]:
input_shapes = [input.shape for input in train_data[0][0]]
n_species = train_data.n_species
print(f"input shape = {input_shapes}")

In [ ]:
n_max_low_occ = 50
low_occ_species = train_data.species_counts[train_data.species_counts <= n_max_low_occ].index
low_occ_species_idx = np.where(np.isin(train_data.species, low_occ_species))[0]
print(f"nb of species with less than {n_max_low_occ} occurrences = {len(low_occ_species_idx)}")

In [ ]:
val_data = MultiScalePatchesDatasetCooccurrences(occurrences=val_occ_path, providers=providers, species=train_data.species)

In [ ]:
batch_size = 2
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(val_data, shuffle=False, batch_size=batch_size)

In [ ]:
modelA = MLP(input_shapes[0][0], 512, 5, 1000)
modelB = get_resnet(512)
model = MultiScaleModel(modelA, modelB, 512, 512, n_species)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
loss_fn = eval(loss)
species_weights = torch.tensor(train_data.species_weights)
val_loss_fn = torch.nn.BCELoss()

In [ ]:
batch = next(iter(train_loader))

In [ ]:
batch[0][0].shape

In [ ]:
(inputsA, inputsB), labels = batch
print(inputsA.shape, inputsB.shape, labels.shape)

In [ ]:
inputsA = inputsA.to(torch.float32)
inputsB = inputsB.to(torch.float32)
out = model(inputsA, inputsB)

In [ ]:
out = torch.sigmoid(out)

In [ ]:
out.shape

In [ ]:
1e-28 + 1e-05 * np.abs(p.nodata_value[0])

In [ ]:
plt.imshow(np.where(np.isclose(p.data, p.nodata_value[0], atol=1e-8), np.nan, p.data)[0,3500:4000,3000:4000])

In [ ]:
p_soil.rasters_providers[0].nodata_value

In [ ]:
for p in p_bioclim.rasters_providers[1:]:
    pseudoabsence_locations += np.where(p.data != 0, 1, 0)


In [ ]:
np.sum(np.where(pseudoabsence_locations != 0, 1, 0))

In [ ]:
5467*8143

In [ ]:
for p in p_bioclim.rasters_providers:
    print(f"{p.name}: xmin={p.x_min} / ymin={p.y_min} / nrows={p.n_rows} / ncold={p.n_cols} / nlayers={p.nb_layers}")

In [ ]:
for p in p_soil.rasters_providers:
    print(f"{p.name}: xmin={p.x_min} / ymin={p.y_min} / nrows={p.n_rows} / ncold={p.n_cols} / nlayers={p.nb_layers}")

In [ ]:
# patch_size = 1
# flatten = True 
# p_bioclim = MultipleRasterPatchProvider(bioclim_dir, size=patch_size, flatten=flatten) 
# p_soil = MultipleRasterPatchProvider(soil_dir, size=patch_size, flatten=flatten) 
# p_landcover = RasterPatchProvider(landcover_dir, size=patch_size, flatten=flatten)
p_sat = JpegPatchProvider(sat_dir, size = 128)

In [ ]:
train_data = PatchesDatasetCooccurrences(occurrences=po_path, providers=(p_sat))
print(f"\nTRAINING DATA: n_items={len(train_data)}, n_species={len(train_data.species)}")
print(train_data[0][0].shape, train_data[0][1].shape)

n_features = train_data[0][0].shape[0]
n_species = len(train_data.species)
print(f"nb of features = {n_features}")

val_data = PatchesDatasetCooccurrences(occurrences=pa_path, providers=(p_sat), species=train_data.species)
print(f"\nVALIDATION DATA: n_items={len(val_data)}, n_species={len(val_data.species)}")
print(val_data[0][0].shape, val_data[0][1].shape)

In [ ]:
low_occ_species = train_data.species_counts[train_data.species_counts < 50].index
low_occ_species

In [ ]:
low_occ_species_idx = np.where(np.isin(train_data.species, low_occ_species))

In [ ]:
low_occ_species_idx[0].shape

In [ ]:
val_loader = torch.utils.data.DataLoader(val_data, shuffle=False, batch_size=1024)
labels_list = []
for inputs, labels in tqdm(val_loader):
    labels_list.append(labels.numpy())
labels = np.concatenate(labels_list)
print(labels.shape)

In [ ]:
labels[:, low_occ_species_idx].shape

In [ ]:
pred = torch.rand(labels.shape).numpy()
pred.shape

In [ ]:
roc_auc_score(labels, pred)

In [ ]:
roc_auc_score(labels[:, low_occ_species_idx], pred[:, low_occ_species_idx])

In [ ]:
.shape

In [ ]:
model = models.resnet18(weights='ResNet18_Weights.IMAGENET1K_V1')
weights = model.conv1.weight.data.clone()
model.conv1 = torch.nn.Conv2d(6, 64, kernel_size=(7,7), stride=(2,2),
                              padding=(3,3), bias=False)
# first three bands are RGB
model.conv1.weight.data[:, :3, :, :] = weights

In [ ]:
model.conv1.weight.data.size()

In [ ]:
model.conv1.weight.data[:, 3:, :, :].size()

In [ ]:
weights[:,0,:,:].repeat((3,1,1,1)).reshape((64,3,7,7))[0,:,:,0]

In [ ]:
weights[0,0,:,0]

In [ ]:
model = models.resnet18(weights='ResNet18_Weights.IMAGENET1K_V1')
weights = model.conv1.weight.data.clone()
model.conv1 = torch.nn.Conv2d(len(p_sat.bands_names), 64, kernel_size=(7,7), stride=(2,2),
                              padding=(3,3), bias=False)
# first three bands are RGB
model.conv1.weight.data[:, :3, :, :] = weights
# initialize weights for 4th band (NIR) to pretrained weights for R
model.conv1.weight.data[:,-1,:,:] = weights[:,0,:,:]
model.fc = torch.nn.Linear(512, n_species)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=100)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
loss_fn = torch.nn.BCELoss()

In [ ]:
for inputs, labels in train_loader:
    print('-----------------------')
    inputs = inputs.to(torch.float32)
    labels = labels.to(torch.float32)

    print(inputs.size())
    print(labels.size())
    pred = model(inputs)
    print(pred.size())
    pred_sigmoid = torch.sigmoid(pred)
    loss = loss_fn(pred_sigmoid, labels)
    print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
img = (img - p_sat.stats[folder]['mean']) / p_sat.stats[folder]['std']
img[0:4,0:4]

In [ ]:
(122-77.3)/25.6
# [[140, 134, 122],
# [130, 124, 110],
# [126, 118, 105],
# [131, 123, 110]]]

# {'rgb': {'mean': array([85.53290863, 89.00972544, 77.32355934]),
#   'std': array([28.46643494, 25.3613768 , 25.57295764])},

In [ ]:

        tensor_list = []
        for folder in set([self.channel_folder[x] for x in self.channels]):
            path = os.path.join(self.root_path, folder, sub_folder_1, sub_folder_2, patch_id+self.ext)
            img = np.asarray(Image.open(path))
            if self.normalize:
                img = (img - self.stats[folder]['mean']) / self.stats[folder]['std']
            if folder == 'rgb':
                img = img.transpose((2,0,1))
            else:
                img = np.expand_dims(img, axis=0)
            tensor_list.append(img)

        tensor = np.concatenate(tensor_list)   

In [ ]:
n_layers=5
width=1000
learning_rate = 1e-3

checkpoint = torch.load(f"models/0129_MLP_env_1x1_an_slds/last.pth") 
model = MLP(n_features, n_species, n_layers, width).to(dev)
model.load_state_dict(checkpoint['state_dict'])

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
batch_size = 100
val_loader = torch.utils.data.DataLoader(val_data, shuffle=False, batch_size=batch_size)

In [ ]:
model.eval()
for inputs, labels in tqdm(val_loader):

    inputs = inputs.to(torch.float32).to(dev)
    labels = labels.to(torch.float32).to(dev) 

    y_pred = model(inputs)
    y_pred_sigmoid = torch.sigmoid(y_pred)
    break

In [ ]:
from util import full_weighted_loss, log_loss, an_slds_loss

In [ ]:
site_loss = ((log_loss(y_pred_sigmoid) * labels) + (log_loss(1 - y_pred_sigmoid) * (1 - labels))).sum(axis=1)
site_loss

In [ ]:
site_weight = 1 / (labels.sum(axis=1) + 1e-5)
site_weight

In [ ]:
(site_loss * site_weight)

In [ ]:
labels.sum(axis=1)

In [ ]:
((log_loss(y_pred_sigmoid) * labels) + (log_loss(1 - y_pred_sigmoid) * (1 - labels))).size()

In [ ]:
torch.transpose((1/labels.sum(axis=1)).repeat((970,1)), 0, 1).size()

In [ ]:
y_pred_sigmoid = torch.sigmoid(y_pred)

In [ ]:
((log_loss(y_pred_sigmoid) * labels) + (log_loss(1 - y_pred_sigmoid) * (1 - labels)))

In [ ]:
per_site_loss = ((log_loss(y_pred_sigmoid) * labels) + (log_loss(1 - y_pred_sigmoid) * (1 - labels))) * torch.transpose((1/labels.sum(axis=1)).repeat((970,1)), 0, 1)

In [ ]:
((log_loss(y_pred_sigmoid) * labels) + (log_loss(1 - y_pred_sigmoid) * (1 - labels))).sum(axis=1).size()

In [ ]:
(1/labels.sum(axis=1)).size()

In [ ]:
site_loss = ((log_loss(y_pred_sigmoid) * labels) + (log_loss(1 - y_pred_sigmoid) * (1 - labels))).sum(axis=1)
site_weight = (1/labels.sum(axis=1))
print(site_loss.size(), site_weight.size())

In [ ]:
(site_loss * site_weight).mean()

In [ ]:
per_site_loss.sum() 

In [ ]:
species_weights = torch.tensor(train_data.species_weights)
species_weights.size()

In [ ]:
(log_loss(pred) * labels).shape# * species_weights.repeat((batch_size, 1))).mean()


In [ ]:
pred * labels

In [ ]:
np.ones(labels.shape[1]).shape

In [ ]:
labels.shape

In [ ]:
species_weights = torch.tensor(train_data.species_weights.values)

In [ ]:
species_weights.repeat((100,1)).size()

In [ ]:
species_weights

In [ ]:
(species_weights/(species_weights-1)).repeat((batch_size,1)).size()

In [ ]:
index = 0 
item_columns=['lat','lon','patchID','dayOfYear']
train_data.items.iloc[index][item_columns].to_dict()

In [ ]:
train_data.items.iloc[index]['speciesId']

In [ ]:
for inputs, labels in tqdm(train_loader):
    print(inputs.shape, labels.shape)

In [ ]:
img.shape

In [ ]:
img[:,0:2,0:2]

In [ ]:
((img - stats[folder]['mean']) / stats[folder]['std'])[:,0:2,0:2]

In [ ]:
img_shape = np.array(Image.open(files[0], mode='r')).shape 
dims = len(img_shape)
dims

In [ ]:
img_shape

In [ ]:
sums = np.array([0.0] * img_shape[2]) if dims == 3 else np.array([0.0])
n_terms = 0

for f in files:
    img = np.array(Image.open(f, mode='r'))
    nans = np.sum(np.isnan(img))
    if nans != 0: 
        print(f)
        break
    img_sum = np.sum(img, axis=(0,1))
    sums += img_sum
    n_terms += img.shape[0] * img.shape[1]

In [ ]:
sums

In [ ]:
means = sums / n_terms
means

In [ ]:
pd.DataFrame({'mean': means}).to_csv('data/sample_data/SatelliteImages/rgb_means.csv', index=False)

In [ ]:
means = pd.read_csv('data/sample_data/SatelliteImages/rgb_means.csv')
means = np.array(means['mean'])
means

In [ ]:
sums = np.array([0.0] * img_shape[2]) if dims == 3 else np.array([0.0])
n_terms = 0

for f in files:
    img = np.array(Image.open(f, mode='r'))
    sum_dev_mean = np.sum(np.power(img - means, 2), axis=(0,1))
    sums += sum_dev_mean
    n_terms += img.shape[0] * img.shape[1]    

In [ ]:
std_devs = np.power(sums / n_terms, 0.5)
std_devs

In [ ]:
pd.DataFrame({'std_dev': std_devs}).to_csv('data/sample_data/SatelliteImages/rgb_std_devs.csv', index=False)

In [ ]:
datadir = 'data/full_data/'
bioclim_dir = datadir+'EnvironmentalRasters/Climate/BioClimatic_Average_1981-2010/'
soil_dir = datadir+'EnvironmentalRasters/Soilgrids/'
po_path = datadir+'Presence_only_occurrences/Presences_only_train_sampled_10_percent_min_100_occurrences.csv' #Presences_only_train_sampled_25_percent_min_10_occurrences.csv'
pa_path = datadir+'Presence_Absence_surveys/Presences_Absences_train.csv'

In [ ]:
occurrences=po_path
label_name='speciesId'
item_columns=['lat','lon','patchID','dayOfYear']
df = pd.read_csv(occurrences, sep=";", header='infer', low_memory=False)
items = pd.DataFrame(df.groupby(item_columns)[label_name].agg(list)).reset_index()

In [ ]:
root_path = sat_dir
ext = '.jpeg'
sub_dirs = next(os.walk(root_path))[1]
select = [x for x in ['rgb','nir'] if x in sub_dirs]
channels = ['red','green','blue'] + [x for x in select if x != 'rgb']
channel_folder = {'red': 'rgb', 'green': 'rgb', 'blue': 'rgb','nir':'nir'}

In [ ]:
item = items.iloc[0][item_columns].to_dict()
patch_id = str(int(item['patchID']))
sub_folder_1 = patch_id[-2:]
sub_folder_2 = patch_id[-4:-2]

In [ ]:
for folder in set([channel_folder[x] for x in channels]):
    path = os.path.join(root_path, folder, sub_folder_1, sub_folder_2, patch_id+ext)
    print(folder, path)
    
    img = np.asarray(Image.open(path))
    if folder == 'rgb':
        img = img.transpose((2,0,1))
    else:
        img = np.expand_dims(img, axis=0)
    
    stats_path = os.path.join(root_path, folder+'_stats.csv')
    print(stats_path)
    stats = pd.read_csv(stats_path, sep=';', index_col=0)
    #mean, std = df.loc[0, 'mean'], df.loc[0, 'std']
                #     img = (img-mean)/std

In [ ]:
stats

In [ ]:
img.shape

In [ ]:
img[:,0:3,0:3]

In [ ]:
for i in range(img.shape[0]):
    img[i,:,:] = img[i,:,:] - stats.loc[i,'mean']

In [ ]:
stats = pd.DataFrame({
    'mean': np.nanmean(imgs,axis=(0,2,3)),
    'std': np.nanstd(imgs,axis=(0,2,3))
})
stats


In [ ]:
re_imgs = imgs.reshape(3,100,128,128)
re_imgs[:, 0, 0:3, 0:3]

In [ ]:
train_data = MultiScalePatchesDatasetCooccurrences(
    occurrences=po_path, 
    providers=((p_bioclim, p_soil))
)

In [ ]:
providers = (p_bioclim, p_soil, p_sat)
patch_sizes = [p.patch_size for p in providers]
patch_sizes

In [ ]:
train_data[0][0]

In [ ]:
unique_patch_sizes = list(set(patch_sizes))
providers_grouped = []
for patch_size in unique_patch_sizes:


In [ ]:
len(train_data[0][0])

In [ ]:
train_data[0][0][0].shape

In [ ]:
train_data[0][0][1].shape

In [ ]:
train_data.n_groups

In [ ]:
batch_size = 10
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)

In [ ]:
for inputs, labels in tqdm(train_loader):
    inputs = inputs#.to(torch.float32)
    labels = labels#.to(torch.float32)
    break

    env = inputs[0].to(torch.float32)
    sat = inputs[1].to(torch.float32)

    print(len(inputs))
    print(labels.shape)

    env_vars = inputs[0]
    sat_vars = inputs[1]
    print(env_vars.shape, sat_vars.shape)

    break
#1m32s for ((p_bioclim, p_soil), (p_sat))

In [ ]:
env = inputs[0].to(torch.float32)
sat = inputs[1].to(torch.float32)
print(env.shape, sat.shape)

In [ ]:
labels.shape

In [ ]:
sat_img = train_data[0][0][1]

In [ ]:
labels = train_data[0][1]

In [ ]:
occurrences=po_path
label_name='speciesId'
item_columns=['lat','lon','patchID','dayOfYear']
df = pd.read_csv(occurrences, sep=";", header='infer', low_memory=False)
species = np.unique(df[label_name].values)
items = pd.DataFrame(df.groupby(item_columns)[label_name].agg(list)).reset_index()
providers = ((p_bioclim, p_soil), (p_sat))

In [ ]:
patch_sizes = []
one_provider_in_group = []
for provider in providers:
    try:
        patch_sizes.append(provider.patch_size)
        one_provider_in_group.append(True)
    except:
        group_patch_sizes = [p.patch_size for p in provider]
        print('group', group_patch_sizes)
        assert len(set(group_patch_sizes)) == 1
        patch_sizes.append(group_patch_sizes[0])
        one_provider_in_group.append(False)
print(patch_sizes)
print(one_provider_in_group)

        

In [ ]:
provider_list = [MetaPatchProvider(p, one_provider=one_p) for p, one_p in zip(providers, one_provider_in_group)]
    

In [ ]:
# get item
item = items.iloc[0][item_columns].to_dict()
item

In [ ]:
patches = []
for p in provider_list:
    patch = p[item]
    print(patch.shape)
    patches.append(patch)

In [ ]:
env = patches[0]
rgb = patches[1]

In [ ]:
for pro in p:
    print(str(pro))

In [ ]:
MetaPatchProvider(p_sat, one_provider=True)

In [ ]:
MetaPatchProvider(providers[1], one_provider=True)

In [ ]:
MetaPatchProvider(providers[0])

In [ ]:
provider_patch_sizes = [p.patch_size for p in providers]
provider_patch_sizes

In [ ]:
unique_patch_sizes = list(set(provider_patch_sizes))
num_patch_sizes = len(unique_patch_sizes)

In [ ]:
for p in unique_patch_sizes:
    idx = list(np.where(np.array(provider_patch_sizes) == p)[0])
    print('--------------------------------------------')
    print(p, idx)
    one_provider = True if len(idx)==1 else False
    meta_p = MetaPatchProvider((providers[i] for i in idx), one_provider)
    print(meta_p)


In [ ]:
root_path = sat_dir
sub_dirs = next(os.walk(root_path))[1]
sub_dirs

In [ ]:
# Dataset
train_data = PatchesDataset(occurrences=po_path, providers=(p_bioclim, p_soil, p_sat))
occurrences=po_path
#providers=(p_bioclim, p_soil)
label_name='speciesId'
item_columns=['lat','lon','patchID','dayOfYear']
df = pd.read_csv(occurrences, sep=";", header='infer', low_memory=False)
species = np.unique(df[label_name].values)
items = df[item_columns + [label_name]]

In [ ]:
index = 0
item = items.iloc[index][item_columns].to_dict()
item

In [ ]:
patch_id = str(int(item['patchID']))
sub_folder_1 = patch_id[-2:]
sub_folder_2 = patch_id[-4:-2]
ext = '.jpeg'
patch_id

In [ ]:
root_path = 'data/sample_data/SatelliteImages/'
channel_folder = {'red': 'rgb', 'green': 'rgb', 'blue': 'rgb','nir':'nir'}

In [ ]:
folder = 'rgb'
path = os.path.join(root_path, folder, sub_folder_1, sub_folder_2, patch_id+ext)
img = np.asarray(Image.open(path)) 
img = img.transpose((2,0,1))
img.shape

In [ ]:
img[0].shape[1]

In [ ]:
list_tensor = {'order': [], 'tensors':[]}
for channel, channel_filename in channels_dict.items():
    print(channel)
    if channel not in list_tensor['order']:
        path = os.path.join(root_path, channel_filename, sub_folder_1, sub_folder_2, patch_id+ext)
        print(path)
        img = np.asarray(Image.open(path))
        print(img.shape)
        if channel in ['red','green','blue']:
            img = img.transpose((2,0,1))
            print(img.shape)

In [ ]:
patch_id = str(int(item['patchID']))
rgb_path = f"{self.rgbnir_dir}rgb/{patch_id[-2:]}/{patch_id[-4:-2]}/{patch_id}.jpeg"
rgb_img = (np.asarray(Image.open(rgb_path)) / 255.0).transpose((2,0,1))

nir_path = f"{self.rgbnir_dir}nir/{patch_id[-2:]}/{patch_id[-4:-2]}/{patch_id}.jpeg"
nir_img = np.expand_dims(np.asarray(Image.open(nir_path)) / 255.0, axis=0)

rgbnir = np.concatenate([rgb_img, nir_img])
if self.rgbnir_patch_size != 128:
    rgbnir = rgbnir[:, round((rgbnir[0].shape[0] - self.rgbnir_patch_size) /2):round((rgbnir[0].shape[0] + self.rgbnir_patch_size) /2),
                                    round((rgbnir[0].shape[1] - self.rgbnir_patch_size) /2):round((rgbnir[0].shape[1] + self.rgbnir_patch_size) /2)]
     

In [ ]:
items[1]

In [ ]:
#class JpegPatchProvider(PatchProvider)
# init


In [ ]:
print(len(train_data.species), train_data.species[0:10])

In [ ]:
label_name='speciesId'
item_columns=['lat','lon','patchID','dayOfYear']
df = pd.read_csv(po_path, sep=";", header='infer', low_memory=False)
species = train_data.species
items = df[item_columns + [label_name]]
items.shape

In [ ]:
items2 = pd.DataFrame(df.groupby(item_columns)[label_name].agg(list)).reset_index()
items2

In [ ]:
counts = df.groupby(item_columns)[label_name].count()
counts

In [ ]:
counts[counts > 5]

In [ ]:
count = 0
for index in range(items2.shape[0]):
    item = items2.iloc[index][item_columns].to_dict()
    item_sps = items2.iloc[index][label_name]
    labels = 1 * np.isin(species, item_sps)
    if np.sum(labels) == 0:
        print('!!!!!!!!', index)
        count += 1


In [ ]:
count

In [ ]:
index = 0
item = items2.iloc[index][item_columns].to_dict()
item

In [ ]:
item_sps = items2.iloc[index][label_name]
print(len(item_sps), item_sps)

In [ ]:
labels = 1 * np.isin(species, item_sps)
labels_f = 1 * np.isin(species, item_sps_f)

In [ ]:
np.all(labels == labels_f)

In [ ]:
print(np.sum(labels), np.sum(labels_f))

In [ ]:
items[~items['speciesId'].isin(species)]

In [ ]:
index = 447
item = items.iloc[index][item_columns].to_dict()
item_species = items.query(
    ' and '.join([f'{k} == {v}' for k, v in item.items()])
)[label_name].values
labels = 1 * np.isin(species, item_species)

In [ ]:
[(s, s in species) for s in item_species]

In [ ]:
print(f"{np.sum([s in species for s in item_species])}/{len(item_species)}")
print(np.sum(labels))

In [ ]:
# print("Making dataset for presence-only training data...")
train_data = PatchesDataset(occurrences=po_path, providers=(p_bioclim, p_soil))
print(f"\nTRAINING DATA: n_items={len(train_data)}, n_species={len(train_data.species)}")

n_features = train_data[0][0].shape[0]
n_species = len(train_data.species)
print(f"nb of features = {n_features}")

In [ ]:
# print("Making dataset for presence-absence validation data...")
val_data = PatchesDataset(occurrences=pa_path, providers=(p_bioclim, p_soil), species=train_data.species)
print(f"\nVALIDATION DATA: n_items={len(val_data)}, n_species={len(val_data.species)}")

In [ ]:
val_data[0][0].shape

In [ ]:
# data loaders
batch_size = 1024
# train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size, num_workers=16)
val_loader = torch.utils.data.DataLoader(val_data, shuffle=False, batch_size=batch_size, num_workers=8)

In [ ]:
run_name = '0116_MLP_env_2x2_flat_train_tinyPO'
checkpoint = torch.load(f"models/{run_name}/best_val_loss.pth")
checkpoint['epoch']

In [ ]:
n_layers = 5
width = 1000
learning_rate = 1e-3

model = MLP(n_features, n_species, n_layers, width)
model.load_state_dict(checkpoint['state_dict'])
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
loss_fn = torch.nn.BCEWithLogitsLoss()

In [ ]:
model.eval()
val_loss_list, labels_list, y_pred_list = [], [], []

for inputs, labels in tqdm(val_loader):
    inputs = inputs.to(torch.float32)
    labels = labels.to(torch.float32)
    labels_list.append(labels.cpu().detach().numpy())

    y_pred = model(inputs)
    y_pred_sigmoid = torch.sigmoid(y_pred)
    y_pred_list.append(y_pred_sigmoid.cpu().detach().numpy())

    val_loss = loss_fn(y_pred, labels)
    val_loss_list.append(val_loss.cpu().detach())    


In [ ]:
labels = np.concatenate(labels_list)
labels.shape

In [ ]:
y_pred = np.concatenate(y_pred_list)
y_pred.shape

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(labels, y_pred)

# patch size = 1            AUC = 0.5725362624834227
# patch size = 2 (flat)     AUC = 0.46973261663879734